In [1]:
import io 
import numpy as np 
import random

import tensorflow as tf
from tensorflow import keras
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [2]:
print('Opening file...')
path = "divina_commedia.txt"
with io.open(path, encoding='utf-8') as file:
    text = file.read().lower()

print("text length", len(text))
print()
print('\n\n\n\n\n', '***** first 1000 characters *****', '\n\n\n\n\n')
text[0:1000]

Opening file...
text length 558240






 ***** first 1000 characters ***** 







"inferno\n\n\n\ninferno: canto i\n\n\nnel mezzo del cammin di nostra vita\n  mi ritrovai per una selva oscura\n  che' la diritta via era smarrita.\n\nahi quanto a dir qual era e` cosa dura\n  esta selva selvaggia e aspra e forte\n  che nel pensier rinova la paura!\n\ntant'e` amara che poco e` piu` morte;\n  ma per trattar del ben ch'i' vi trovai,\n  diro` de l'altre cose ch'i' v'ho scorte.\n\nio non so ben ridir com'i' v'intrai,\n  tant'era pien di sonno a quel punto\n  che la verace via abbandonai.\n\nma poi ch'i' fui al pie` d'un colle giunto,\n  la` dove terminava quella valle\n  che m'avea di paura il cor compunto,\n\nguardai in alto, e vidi le sue spalle\n  vestite gia` de' raggi del pianeta\n  che mena dritto altrui per ogne calle.\n\nallor fu la paura un poco queta\n  che nel lago del cor m'era durata\n  la notte ch'i' passai con tanta pieta.\n\ne come quei che con lena affannata\n  uscito fuor del pelago a la riva\n  si volge a l'acqua perigliosa e guata,\n\ncosi` l'animo mio, 

In [3]:
with io.open("vita_nova.txt", encoding='utf-8') as file:
    external_text = file.read().lower()

print("'vita nova' text length", len(external_text))
print()
print('\n\n\n\n\n', '***** first 1000 characters *****', '\n\n\n\n\n')
external_text[0:1000]

'vita nova' text length 104284






 ***** first 1000 characters ***** 







'i\nin quella parte del libro de la mia memoria, dinanzi a la quale poco si potrebbe leggere, si trova una rubrica la quale dice: incipit vita nova. sotto la quale rubrica io trovo scritte le parole le quali è mio intendimento d’asemplare in questo libello; e se non tutte, almeno la loro sentenzia.\n\nii\n[i] nove fiate già appresso lo mio nascimento era tornato lo cielo de la luce quasi a uno medesimo punto, quanto a la sua propria girazione, quando a li miei occhi apparve prima la gloriosa donna de la mia mente, la quale fu chiamata da molti beatrice, li quali non sapeano che si chiamare. ella era in questa vita già stata tanto, che ne lo suo tempo lo cielo stellato era mosso verso la parte d’oriente de le dodici parti l’una d’un grado, sì che quasi dal principio del suo anno nono apparve a me, ed io la vidi quasi da la fine del mio nono. apparve vestita di nobilissimo colore, umile ed onesto, sanguigno, cinta e ornata a la guisa che a la sua giovanissima etade si convenia. in quello

In [4]:
chars = sorted(list(set(text) | set(external_text)))
print('total chars: ', len(chars))

char_indices = dict((c, i) for i, c in enumerate(chars)) # create first dictionary
indices_char = dict((i, c) for i, c in enumerate(chars))

print(char_indices)
print(indices_char)

total chars:  54
{'\n': 0, ' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, ',': 7, '-': 8, '.': 9, ':': 10, ';': 11, '<': 12, '>': 13, '?': 14, '[': 15, ']': 16, '`': 17, 'a': 18, 'b': 19, 'c': 20, 'd': 21, 'e': 22, 'f': 23, 'g': 24, 'h': 25, 'i': 26, 'j': 27, 'l': 28, 'm': 29, 'n': 30, 'o': 31, 'p': 32, 'q': 33, 'r': 34, 's': 35, 't': 36, 'u': 37, 'v': 38, 'x': 39, 'y': 40, 'z': 41, '«': 42, '»': 43, 'à': 44, 'â': 45, 'è': 46, 'é': 47, 'ê': 48, 'ì': 49, 'ò': 50, 'ù': 51, '‘': 52, '’': 53}
{0: '\n', 1: ' ', 2: '!', 3: '"', 4: "'", 5: '(', 6: ')', 7: ',', 8: '-', 9: '.', 10: ':', 11: ';', 12: '<', 13: '>', 14: '?', 15: '[', 16: ']', 17: '`', 18: 'a', 19: 'b', 20: 'c', 21: 'd', 22: 'e', 23: 'f', 24: 'g', 25: 'h', 26: 'i', 27: 'j', 28: 'l', 29: 'm', 30: 'n', 31: 'o', 32: 'p', 33: 'q', 34: 'r', 35: 's', 36: 't', 37: 'u', 38: 'v', 39: 'x', 40: 'y', 41: 'z', 42: '«', 43: '»', 44: 'à', 45: 'â', 46: 'è', 47: 'é', 48: 'ê', 49: 'ì', 50: 'ò', 51: 'ù', 52: '‘', 53: '’'}


In [5]:
maxlen = 30 # chunk length
step = 3

sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

print('number of sequences: ', len(sentences))
print(sentences[11])
print(next_chars[11])

number of sequences:  186070
 mezzo del cammin di nostra vi
t


In [6]:
# first point of assignment
from sklearn.model_selection import train_test_split

sentences_train, sentences_val, next_chars_train, next_chars_val = train_test_split(
    sentences, next_chars, test_size=0.2, random_state=42
)

print(f"Training set: {len(sentences_train)}")
print(f"Validation set: {len(sentences_val)}")

Training set: 148856
Validation set: 37214


In [7]:
# encode in one rapresentation
print('generating input and output..')

def one_hot_encoding(sentences, next_chars, maxlen, chars, char_indices): 
    x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
    y = np.zeros((len(sentences), len(chars)), dtype=bool)

    for i, sentence in enumerate(sentences):
        for t, char in enumerate(sentence):
            x[i, t, char_indices[char]] = 1
        y[i, char_indices[next_chars[i]]] = 1

    return x, y

generating input and output..


In [9]:
import sys

results = {}

# Possibili valori di maxlen per il tuning
maxlen_values = [40, 80, 120]

# Ciclo attraverso i valori di maxlen
for maxlen in maxlen_values:

    # Codifica i dati con il nuovo maxlen
    x_train, y_train = one_hot_encoding(sentences, next_chars, maxlen, chars, char_indices)
    x_val, y_val = one_hot_encoding(sentences, next_chars, maxlen, chars, char_indices)
    
    # Crea e compila il modello
    model = Sequential()
    model.add(LSTM(128, input_shape=(maxlen, len(chars))))
    model.add(Dense(len(chars), activation='softmax'))

    optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01)

    # second point
    model.compile(
        loss='categorical_crossentropy', 
        optimizer=optimizer, 
        metrics=['accuracy']
    )

    model.summary()

    def testAfterEpoch(epoch, _):
        print()
        print()
        print('***** Epoch: %d *****' % (epoch+1))
        start_index = random.randint(0, len(text)- maxlen - 1)

        generated = ''
        sentence = text[start_index : start_index + maxlen]
        generated = generated + sentence

        print('***** starting sentence *****') 
        print(sentence)
        print('*****************************')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1
            
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = np.argmax(preds)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

    print_callback = LambdaCallback(on_epoch_end=testAfterEpoch)

    # Allena il modello
    model.fit(x_train, y_train,
          batch_size = 2048, 
          epochs = 20, 
          callbacks = [print_callback],
          validation_data =(x_val, y_val))

    # Valuta il modello
    test_loss, test_accuracy = model.evaluate(x_val, y_val, verbose=1)
    
    print(f"Maxlen: {maxlen} -> Validation Loss: {test_loss}, Validation Accuracy: {test_accuracy}")

    # Puoi anche conservare i risultati in un dizionario per un confronto più facile
    results[maxlen] = {'loss': test_loss, 'accuracy': test_accuracy}

# Alla fine, potresti stampare i risultati per ogni maxlen:
print("\n\nSummary of Results:")
for maxlen, metrics in results.items():
    print(f"Maxlen: {maxlen}, Loss: {metrics['loss']}, Accuracy: {metrics['accuracy']}")


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                   │ (None, 128)            │        93,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 54)             │         6,966 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 100,662 (393.21 KB)

 Trainable params: 100,662 (393.21 KB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_5 (LSTM)                   │ (None, 128)            │        93,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 54)             │         6,966 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 100,662 (393.21 KB)

 Trainable params: 100,662 (393.21 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1628 - loss: 3.1509

***** Epoch: 1 *****
***** starting sentence *****
 oltracotanza in voi s'alletta?

perche' recalcitrate a quella voglia
  a cui non puote il fin mai esser mozzo,
  e che 
*****************************
 oltracotanza in voi s'alletta?

perche' recalcitrate a quella voglia
  a cui non puote il fin mai esser mozzo,
  e che                                                                                                                                                                                                                                                                                                                                                                                                                 
91/91 ━━━━━━━━━━━━━━━━━━━━ 216s 2s/step - accuracy: 0.1629 - loss: 3.1495 - val_accuracy: 0.1806 - val_loss: 2.9712
Epoch 2/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1824 - loss: 2